In [ ]:
import numpy as np
import datetime
import xarray as xr

from tools.utils import create_polydict_ptsarr, xy2ll, ll2xy
from tools.landsat_utils import easy_raster, corr_lakes_mask, normalize_band as normalize

import matplotlib.pyplot as plt
from tools.plot_tools import plot_latlon_lines
from matplotlib import gridspec

In [ ]:
import plotSettings
plotSettings.paper()
COLS = plotSettings.COLS

In [ ]:
%matplotlib tk

In [ ]:
# general data directory
data_dir = '/'

In [ ]:
center_lon, center_lat = 69.5881, -72.0370
center_x, center_y = ll2xy(center_lon, center_lat)
# needed for RACMO data
center_rlon, center_rlat = 16.08, 8.93 

# climate-related 
RACMO_dir = 'data/RACMO2.3'
RACMO_dir = os.path.join(data_dir, RACMO_dir)
RACMO_files = [os.path.join(RACMO_dir,f) for f in os.listdir(RACMO_dir) if '.nc' in f]

# surface melt image files
years = [str(y) for y in range(2015-3, 2015+4)]
# selected
surface_melt_img_dirs = [
# 2013
    'data/LandSat8-9_Color/LC08_L1GT_126111_20131109_20201016_02_T2',
# 2014
    'data/LandSat8-9_Color/LC08_L1GT_128111_20140227_20201016_02_T2',
# 2015
    'data/LandSat8-9_Color/LC08_L1GT_128111_20150403_20200909_02_T2',
# 2016
    'data/LandSat8-9_Color/LC08_L1GT_127111_20160226_20201016_02_T2',
# 2017
    'data/LandSat8-9_Color/LC08_L1GT_127111_20170228_20201016_02_T2'
]

# list of all landsat8-9 images
l89_imgs = [os.path.join('data/LandSat8-9_Color',d) for d in os.listdir(os.path.join(data_dir, 'data/LandSat8-9_Color')) if 'LC' in d and '.tar' not in d and '._' not in d]
# melt quantification data
melt_quant_file = 'data/AmeryIS_LS8meltquant_sum.nc'

# CATS2008 tidal
tidal_amp_file = 'data/AmeryIS_tides_CATS2008_2015-02-01_2015-05-01.nc'

# bounding boxes for masking images 
bounding_box_file = 'GIS/BoundingBoxes2.shp'
bbs_dict, _ = create_polydict_ptsarr(os.path.join(data_dir, bounding_box_file))
bb = bbs_dict[0]

In [ ]:
x, corr, moussavi = [], [], []

for dirn in l89_imgs:
    print(dirn)
    d = easy_raster(os.path.join(data_dir, dirn), 
                    bands=['B2', 'B3', 'B4', 'B5', 'B6', 'B8', 'B10'],
                    mask=bbs_dict[4],
                    always_TOA=True)
    ps = d.pan_sharpen('IHS', TOA_correction=True)
    # corr+ 2022 method
    water, clouds, shadows, lake_mask = corr_lakes_mask(ps)
    
    # moussavie method (default)
    mlake_mask = d.lake_mask()
    print(d.date, np.sum(lake_mask), np.sum(mlake_mask))
    x.append(d.date)
    corr.append(np.sum(lake_mask))
    moussavi.append(np.sum(mlake_mask))
    
    print('')


mq = xr.Dataset(data_vars=dict(corr=(['time'],corr), moussavi=(['time'],moussavi)),
                                 coords=dict(time=x)
                                )
mq = mq.sortby('time')
mq.to_netcdf(os.path.join(data_dir, melt_quant_file))


In [ ]:
end_SA_year = 2022

print('loading RACMO data')
t2m = xr.open_dataset([f for f in RACMO_files if 't2m' in f][0]).sel(rlon=center_rlon, rlat=center_rlat, method='nearest').isel(height=0)
snowmelt = xr.open_dataset([f for f in RACMO_files if 'snowmelt' in f][0]).sel(rlon=center_rlon, rlat=center_rlat, method='nearest').isel(height=0)
smb = xr.open_dataset([f for f in RACMO_files if 'smb' in f][0]).sel(rlon=center_rlon, rlat=center_rlat, method='nearest').isel(height=0)
tskin = xr.open_dataset([f for f in RACMO_files if 'tskin' in f][0]).sel(rlon=center_rlon, rlat=center_rlat, method='nearest').isel(height=0)
wind_speed = xr.open_dataset([f for f in RACMO_files if 'ff10m' in f][0]).sel(rlon=center_rlon, rlat=center_rlat, method='nearest').isel(height=0)
RACMO = xr.Dataset(data_vars=dict(t2m=(['time'], t2m.t2m.values),
                                  tskin=(['time'], tskin.tskin.values),
                                  ff10=(['time'], wind_speed.ff10m.values),
                                  snowmelt=(['time'], snowmelt.snowmelt.values),
                                  smb=(['time'], smb.smb.values)),
                   coords=dict(time=xr.date_range(f"{t2m['time.year'][0]}-1-1",f"{t2m['time.year'][-1]}-12-1", freq="1MS"))
                
               )
print('  calculating monthly standardized anomaly')
RSA = xr.apply_ufunc(
    lambda x,m,s: (x-m)/s,
    RACMO.groupby('time.month'),
    RACMO.sel(time=(RACMO['time.year'] <= end_SA_year)).groupby('time.month').mean(),
    RACMO.sel(time=(RACMO['time.year'] <= end_SA_year)).groupby('time.month').std(),               
    dask='allowed'
)


print('loading melt quantification')
melt_quantification = xr.open_dataset(os.path.join(data_dir, melt_quant_file))

print('loading tidal data')
tidal_data = xr.open_dataset(os.path.join(data_dir, tidal_amp_file)).sel(x=center_x, y=center_y, method='nearest')

In [ ]:
SAVE = False
save_name = 'figure4'
save_dir = 'figs'
save_type = '.fig'

############################################################################################################
# plot settings
############################################################################################################
NROWS = 3
NCOLS = 5
figsize = [12.88533333,  7.86133333]
num_months = 3
num_years = 1
collapse_timerange = [datetime.datetime(2015,3,29), datetime.datetime(2015,4,3)]
timeseries_range = [datetime.datetime(2013,9,1),datetime.datetime(2017,4,3)]
timeseries_range = xr.date_range(timeseries_range[0], timeseries_range[-1])
years = [y for y in range(2015 - num_years, 2015 + num_years + 1)]
grounding_line_opts = {'color': 'k', 'lw': 0.5, 'ls':'-'}

############################################################################################################
# create figure and all subplots
############################################################################################################

fig = plt.figure(num=2,clear=1, figsize=figsize)

PARENT = gridspec.GridSpec(NROWS, NCOLS, figure=fig, 
                          hspace=0.15,height_ratios=[1.,1.1,1])
gs0 = gridspec.GridSpecFromSubplotSpec(1,1, subplot_spec=PARENT[0,:])
climax = fig.add_subplot(gs0[0])

gs1 = gridspec.GridSpecFromSubplotSpec(1,NCOLS, subplot_spec=PARENT[1,:], wspace=0.05)
smaxs = [fig.add_subplot(gs1[i]) for i in range(NCOLS)]

gs2 = gridspec.GridSpecFromSubplotSpec(1,1, subplot_spec=PARENT[2,:])
myrax = fig.add_subplot(gs2[0])

############################################################################################################
# climate data plotting
############################################################################################################

print('now plotting climate forcing')
climax.plot(RACMO.sel(time=timeseries_range, method='nearest').time.values, 
            RACMO.sel(time=timeseries_range, method='nearest').t2m, 
             ls='--', marker='o',
            )

climax.axvspan(collapse_timerange[0], collapse_timerange[1],
                fc='k', ec='none',alpha=0.33)
climax.set(xlim=(timeseries_range[0], timeseries_range[-1]))
climax.tick_params(axis='y', labelcolor=COLS[0])
climax.set_ylabel('Temperature 2m (K)', color=COLS[0])

# standardized anomaly
ax2 = climax.twinx()
ax2.plot(RSA.sel(time=timeseries_range, method='nearest').time.values, 
         RSA.sel(time=timeseries_range, method='nearest').t2m, 
         c=COLS[1], marker='s', ls='--')
             
ax2.axhline(0, ls='--', c='k')
ax2.tick_params(axis='y', labelcolor=COLS[1])
ax2.set_ylabel('Standardized Anomaly', color=COLS[1])

############################################################################################################
# surface melt images
############################################################################################################
print('now plotting surface melt')
letters = ['b', 'c', 'd', 'e', 'f'] # subplot labels
smax_dates = []
for i, file_dir in enumerate(surface_melt_img_dirs):
    print(f'\t {i+1}/{len(surface_melt_img_dirs)}: loading', file_dir[-40:])
    rast = easy_raster(os.path.join(data_dir, file_dir), 
                       bands=['B4', 'B3', 'B2', 'B8', 'B5', 'B6', 'B10'],
                       mask=bbs_dict[1],
                       always_TOA=True)
    left, bottom, right, top = bbs_dict[1].bounds
    print('\t\t pan sharepning')
    ps = rast.pan_sharpen(method='brovey', TOA_correction=True)
    R,G,B = ps['B4'], ps['B3'], ps['B2']
    MIN = (0,0,0)
    MAX = [np.quantile(b, 0.98) for b in [R,G,B]]
    shape = R.shape
    RGB = np.empty((shape[0], shape[1], 3),dtype=int)
    RGB[:,:,0] = 255*(normalize(R, MIN[0], MAX[0]))
    RGB[:,:,1] = 255*(normalize(G, MIN[1], MAX[1]))
    RGB[:,:,2] = 255*(normalize(B, MIN[2], MAX[2]))
    
    smaxs[i].imshow(RGB, extent=(left, right, bottom, top), rasterized=True, aspect='equal')

    print('\t\t masking lakes')
    rast = easy_raster(os.path.join(data_dir, file_dir), 
                       bands=['B4', 'B3', 'B2', 'B8', 'B5', 'B6', 'B10'],
                       mask=bbs_dict[4],
                       always_TOA=True)
    LM = rast.lake_mask()
    x = np.linspace(left, right, LM.shape[1])
    y = np.linspace(bottom, top, LM.shape[0])
    smaxs[i].contour(x, y[::-1], LM, 
                     levels=[0, 0.5, 1], colors=['b'], linewidths=[0.5])
    
    smaxs[i].plot(bbs_dict[4].exterior.xy[0], bbs_dict[4].exterior.xy[1], **grounding_line_opts)
    for intr in bbs_dict[4].interiors:
        smaxs[i].plot(intr.xy[0], intr.xy[1], **grounding_line_opts)
    
    yy,mm,dd = rast.date.year, rast.date.month, rast.date.day
    smax_dates.append(rast.date)
    smaxs[i].set(xticklabels=[], yticklabels=[],
                 xticks=[], yticks=[],
                 xlim=(left, right), ylim=(bottom, top),
                )

    smaxs[i].text(left + 1000, bottom + 1000, 
                  f'({letters[i]}) {yy}-{mm}-{dd}',
                  horizontalalignment='left')
    
    ll=xy2ll(left, bottom), xy2ll(left, top), xy2ll(right, top), xy2ll(right, bottom)
    ll=np.array(ll)
    lon_range = [ll[:,0].min(), ll[:,0].max()]
    lat_range = [ll[:,1].min(), ll[:,1].max()]
    print('\t\t plotting lat lon lines')
    smaxs[i] = plot_latlon_lines(smaxs[i], lat_range=lat_range, lon_range=lon_range,
                                 lat_step=0.5, lon_step=1,
                                 extents=(left, right, bottom, top),
                                 lon_text=False if i>0 else True)
    smaxs[i].plot(bb.exterior.xy[0], bb.exterior.xy[1], c=COLS[1])
    [i.set_linewidth(1) for _,i in smaxs[i].spines.items()]
    
############################################################################################################   
# melt yearly compare
############################################################################################################
print('plotting yearly melt comparison')

for year in range(timeseries_range[0].year - 1,2022):
    dr = xr.date_range(f'{year}-11-1',f'{year+1}-5-1')
    myrax.plot(RACMO.sel(time=dr,method='nearest').time,
             RACMO.sel(time=dr,method='nearest').cumulative_integrate('time','s').snowmelt/(3600 * 24 * 365.25/12),
             'o--', c=COLS[0])
    
myrax.plot([np.nan]*2, [np.nan]*2, 'o--', c=COLS[0], label='RACMO cumulative\nsnowmelt')
myrax.scatter(melt_quantification.time,
              melt_quantification.moussavi, 
              marker='s', c=COLS[0], label='# water-filled pixels')

myrax.axvspan(collapse_timerange[0], collapse_timerange[-1], 
              fc='k', ec='none',alpha=0.33)

myrax.set(xlim=(timeseries_range[0], timeseries_range[-1]),
          yscale='symlog')
myrax.tick_params(axis='y', labelcolor=COLS[0])
myrax.legend()

# now plot the tidal forcing 
tideax = myrax.twinx()
dr = xr.date_range('2015-2-1', '2015-5-1')
tideax.plot(tidal_data.sel(time=dr, method='nearest').time, 
            tidal_data.sel(time=dr, method='nearest').dayrange, 
            c='k')
tideax.set_ylabel('Daily tidal amplitude (m)')


[[ax.axvline(d,c=COLS[2]) for d in smax_dates] for ax in [climax, myrax]]#, flax[1]]]

PARENT.update(left=0.06,right=0.94,
              bottom=0.025, top=0.975
             )
PARENT.tight_layout(fig)

if SAVE:
    plt.savefig(os.path.join(save_dir, save_name + fig_type), dpi=600)